In [1]:
import cv2
import numpy as np
import HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
wCam, hCam = 640,480
cam = cv2.VideoCapture(0)
cam.set(3,wCam)
cam.set(4,hCam)
detector = htm.handDetector(detectionCon=0.6)

##############################

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
##############################

volBar = 400
volPer =0

while True:
    _ , img = cam.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img,draw=False)
    if len(lmList)!=0:
        x1,y1 = lmList[4][1],lmList[4][2]
        x2,y2 = lmList[8][1],lmList[8][2]
        cx,cy = (x1+x2)//2 , (y1+y2)//2
        
        cv2.circle(img,(x1,y1),10,(255,0,255),cv2.FILLED)
        cv2.circle(img,(x2,y2),10,(255,0,255),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        cv2.circle(img,(cx,cy),10,(255,0,255),cv2.FILLED)
        lenght = math.hypot(x2-x1,y2-y1)
        
        #Hand range 30 to 150
        #Volume range -65 to 0
        vol = np.interp(lenght,[30,150],[minVol,maxVol])
        volBar = int(np.interp(lenght,[30,150],[400,150]))
        volPer = int(np.interp(lenght,[30,150],[0,100]))
        volume.SetMasterVolumeLevel(vol, None)
        if lenght<30:
            cv2.circle(img,(cx,cy),10,(0,255,0),cv2.FILLED)
        if lenght>150:
            cv2.circle(img,(cx,cy),10,(0,0,255),cv2.FILLED)
            
    cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(img,(50,volBar),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(img,str(volPer),(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),0)       
        
    
    
    cv2.imshow("img",img)
    if cv2.waitKey(1) ==13:
        break
cv2.destroyAllWindows()
cam.release()